The code for this is found: https://sdmx1.readthedocs.io/en/v2.22.0/sources.html

# API

In [1]:
import sdmx
import pandas as  pd 
client = sdmx.Client()
url = (
    # Base URL
    "http://dataservices.imf.org/REST/SDMX_XML.svc/CompactData/"
    # Data flow ID and key
    "DOT/M..TMG_CIF_USD.US+CN+B0"
    # Query parameters, including format
    "?startPeriod=2000&format=sdmx-2.1"
)

In [2]:
def get_highest_source(row):
    sources = {'B0': row['B0'], 'CN': row['CN'], 'US': row['US']}
    # Remove NaN values
    valid_sources = {k: v for k, v in sources.items() if pd.notna(v)}
    
    if not valid_sources:
        return None, None  # All values are NaN
    
    total = sum(valid_sources.values())
    max_key = max(valid_sources, key=valid_sources.get)
    max_value = valid_sources[max_key]
    
    return max_key, round(((max_value / total)*100), 1), round(max_value, 1)



def imf_data(url):
    message = client.get(url=url)
    raw = sdmx.to_pandas(message.data[0])
        
    process_1 = raw.reset_index()
    
    process_2 = process_1.pivot_table(
        index=['REF_AREA', 'TIME_PERIOD'], 
        columns='COUNTERPART_AREA',  
        values='value', 
        aggfunc='first')
    
    process_2.reset_index(inplace=True)
    process_2.columns.name = None

    process_3 = process_2.rename(columns={
        'FREQ': 'Data Granularity',
        'REF_AREA': 'Importer_Code',
        'INDICATOR': 'Indicator',
        'COUNTERPART_AREA': 'Exporter_Code',
    })

    process_3[['Import Partner', 'Percent', 'Amount (USD Millions)']] = process_3.apply(lambda row: pd.Series(get_highest_source(row)), axis=1)
    

    process_3['Import Partner'] = process_3['Import Partner'].map({'B0': 'European Union', 'US': 'United States', 'CN': 'China'})
    process_3[['Year', 'Month']] = process_3['TIME_PERIOD'].str.split('-', expand=True)

    

    return process_3


In [3]:
data = imf_data(url)

xml.Reader got no structure=… argument for StructureSpecificTimeSeriesData


In [4]:
data.head()

,Importer_Code,TIME_PERIOD,B0,CN,US,Import Partner,Percent,Amount (USD Millions),Year,Month
0,1C_080,2000-01,3614.345317,449.412672,1642.461369,European Union,63.3,3614.3,2000,01
1,1C_080,2000-02,3885.422689,540.175471,1853.894880,European Union,61.9,3885.4,2000,02
2,1C_080,2000-03,4661.890084,634.667641,2186.011737,European Union,62.3,4661.9,2000,03
3,1C_080,2000-04,3616.650800,507.603242,1736.763672,European Union,61.7,3616.7,2000,04
4,1C_080,2000-05,4310.082377,612.838257,1754.597474,European Union,64.5,4310.1,2000,05


## Visualizing

In [6]:
tk = data

tk.head()

,Importer_Code,TIME_PERIOD,B0,CN,US,Import Partner,Percent,Amount (USD Millions),Year,Month
0,1C_080,2000-01,3614.345317,449.412672,1642.461369,European Union,63.3,3614.3,2000,01
1,1C_080,2000-02,3885.422689,540.175471,1853.894880,European Union,61.9,3885.4,2000,02
2,1C_080,2000-03,4661.890084,634.667641,2186.011737,European Union,62.3,4661.9,2000,03
3,1C_080,2000-04,3616.650800,507.603242,1736.763672,European Union,61.7,3616.7,2000,04
4,1C_080,2000-05,4310.082377,612.838257,1754.597474,European Union,64.5,4310.1,2000,05


In [7]:
import requests

In [8]:
geojson_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'
response = requests.get(geojson_url)
geojson = response.json()

In [9]:
import pycountry

def iso2_to_iso3(code):
    try:
        return pycountry.countries.get(alpha_2=code).alpha_3
    except:
        return None  # Invalid or custom code

# Apply conversion
tk['ISO3'] = tk['Importer_Code'].apply(iso2_to_iso3)

# Drop rows where ISO3 conversion failed
tk = tk.dropna(subset=['ISO3'])


In [20]:
tk.iloc[:,5]

1510     European Union
1511     European Union
1512     European Union
1513     European Union
1514     European Union
              ...      
67896             China
67897             China
67898             China
67899             China
67900             China
Name: Import Partner, Length: 62061, dtype: object

In [23]:
source_mapping = {
    'China': 1,
    'European Union': 2, 
    'United States': 3
    
}
tk.loc[:, 'source_cat'] = tk['Import Partner'].map(source_mapping)


In [25]:
def bin_share(value):
    if value <= 25:
        return 0
    elif value <= 50:
        return 1
    elif value <= 75:
        return 2
    else:
        return 3

tk.loc[:, 'share_bin'] = tk['Percent'].apply(bin_share).copy()

In [26]:
tk.head()

,Importer_Code,TIME_PERIOD,B0,CN,US,Import Partner,Percent,Amount (USD Millions),Year,Month,ISO3,source_cat,share_bin
1510,AE,2000-01,583.336060,116.085468,149.379408,European Union,68.7,583.3,2000,01,ARE,2,2
1511,AE,2000-02,557.372105,123.654104,144.058312,European Union,67.6,557.4,2000,02,ARE,2,2
1512,AE,2000-03,735.878667,177.623174,191.747882,European Union,66.6,735.9,2000,03,ARE,2,2
1513,AE,2000-04,542.492052,140.165411,142.626107,European Union,65.7,542.5,2000,04,ARE,2,2
1514,AE,2000-05,654.974195,178.418278,173.542501,European Union,65.0,655.0,2000,05,ARE,2,2


In [29]:


tk_lookup = tk.set_index('ISO3')[['Import Partner', 'Percent', 'Amount (USD Millions)']].to_dict(orient='index')

# Enrich geojson features
for feature in geojson['features']:
    iso = feature['id']
    if iso in tk_lookup:
        lookup = tk_lookup[iso]
        feature['properties']['Import Partner'] = lookup.get('Import Partner', 'N/A')
        feature['properties']['Percent'] = lookup.get('Percent', 'N/A')
        feature['properties']['Amount (USD Millions)'] = lookup.get('Amount (USD Millions)', 'N/A')
    else:
        feature['properties']['Import Partner'] = 'N/A'
        feature['properties']['Percent'] = 'N/A'
        feature['properties']['Amount (USD Millions)'] = 'N/A'


ValueError: DataFrame index must be unique for orient='index'.

In [27]:
import folium

# Step 3: Define color grid (3 sources × 4 bins)
color_grid = {
    (1, 0): '#ffcccc', (1, 1): '#ff9999', (1, 2): '#ff6666', (1, 3): '#cc0000',  # Red shades
    (2, 0): '#ffffcc', (2, 1): '#ffff99', (2, 2): '#ffff66', (2, 3): '#cccc00',  # Yellow shades
    (3, 0): '#ccccff', (3, 1): '#9999ff', (3, 2): '#6666ff', (3, 3): '#0000cc'   # Blue shades
}

# Step 4: Map each country to a color
tk['color_key'] = tk.apply(lambda row: color_grid.get((row['source_cat'], row['share_bin']), '#cccccc'), axis=1)
color_dict = tk.set_index('ISO3')['color_key'].to_dict()

# Step 5: Create Folium map
M = folium.Map(location=[20, 10], zoom_start=2)

# Styling function
def style_function(feature):
    iso = feature['id']  # This assumes GeoJSON Feature has 'id' as ISO3
    return {
        'fillColor': color_dict.get(iso, '#cccccc'),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.8
    }

# Add GeoJson to the map
folium.GeoJson(
    geojson,
    style_function=style_function,
    tooltip = folium.GeoJsonTooltip(
        fields=['name', 'Import Partner', 'Percent', 'Amount (USD Millions)'],
        aliases=['Country', 'Top Import Source', 'Percent', 'Amount (USD Millions)'],
        localize=True
)

).add_to(M)

# Show map
M

C:\Users\rafid\AppData\Local\Temp\ipykernel_22412\3559833442.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tk['color_key'] = tk.apply(lambda row: color_grid.get((row['source_cat'], row['share_bin']), '#cccccc'), axis=1)


AssertionError: The field Import Partner is not available in the data. Choose from: ('name',).

In [33]:
# Dropdown widgets for Year and Month
year_dropdown = widgets.Dropdown(
    options=sorted(tk['Year'].unique()),
    description='Year:',
    style={'description_width': 'initial'}
)

month_dropdown = widgets.Dropdown(
    options=sorted(tk['Month'].unique()),
    description='Month:',
    style={'description_width': 'initial'}
)

# Output area for the map
map_output = widgets.Output()

# Function to update the map based on dropdowns
def update_map(change=None):
    with map_output:
        clear_output()

        # Filter data based on selections
        filtered_tk = tk[(tk['Year'] == year_dropdown.value) & (tk['Month'] == month_dropdown.value)]

        # Rebuild color dict
        filtered_tk.loc[:,'color_key'] = filtered_tk.apply(
            lambda row: color_grid.get((row['source_cat'], row['share_bin']), '#cccccc'), axis=1
        )
        color_dict = filtered_tk.set_index('ISO3')['color_key'].to_dict()

        # Update geojson with tooltip data
        tk_lookup = filtered_tk.set_index('ISO3')[['Import Partner', 'Percent', 'Amount (USD Millions)']].to_dict(orient='index')
        for feature in geojson['features']:
            iso = feature['id']
            if iso in tk_lookup:
                feature['properties']['Import Partner'] = tk_lookup[iso]['Import Partner']
                feature['properties']['Percent'] = tk_lookup[iso]['Percent']
                feature['properties']['Amount (USD Millions)'] = tk_lookup[iso]['Amount (USD Millions)']
            else:
                feature['properties']['Import Partner'] = 'N/A'
                feature['properties']['Percent'] = 'N/A'
                feature['properties']['Amount (USD Millions)'] = 'N/A'

        # Style function
        def style_function(feature):
            iso = feature['id']
            return {
                'fillColor': color_dict.get(iso, '#cccccc'),
                'color': 'black',
                'weight': 0.5,
                'fillOpacity': 0.8
            }

        # Create map
        M = folium.Map(location=[20, 10], zoom_start=2)
        folium.GeoJson(
            geojson,
            style_function=style_function,
            tooltip=folium.GeoJsonTooltip(
                fields=['name', 'Import Partner', 'Percent', 'Amount (USD Millions)'],
                aliases=['Country', 'Top Import Source', 'Percent', 'Amount (USD Millions)'],
                localize=True
            )
        ).add_to(M)

        display(M)

# Set event handlers
year_dropdown.observe(update_map, names='value')
month_dropdown.observe(update_map, names='value')

# Initial display
display(widgets.HBox([year_dropdown, month_dropdown]))
display(map_output)

# Trigger first map
update_map()

Output()

In [46]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import folium

year_month_options = sorted(tk['TIME_PERIOD'].unique())

# Create the index-based slider
slider = widgets.SelectionSlider(
    options=year_month_options,
    description='Period:',
    orientation='horizontal',
    layout={'width': '90%'},
    style={'description_width': 'initial'}
)

# Loading indicator
loading_indicator = widgets.HTML(
    value="",
    placeholder="Loading...",
    description=""
)

# Output area for the map
map_output = widgets.Output()

# Function to update the map based on slider
def update_map(change=None):
    with map_output:
        clear_output()
        
        # Parse year and month from slider value (no clearing output yet)
        year_month = slider.value
        year = int(year_month.split('-')[0])
        month = int(year_month.split('-')[1])

        # Filter data based on selections with more robust comparison
        filtered_tk = tk[(tk['Year'].astype(int) == year) & (tk['Month'].astype(int) == month)]

        # Rebuild color dict
        filtered_tk.loc[:,'color_key'] = filtered_tk.apply(
            lambda row: color_grid.get((row['source_cat'], row['share_bin']), '#cccccc'), axis=1
        )
        color_dict = filtered_tk.set_index('ISO3')['color_key'].to_dict()

        # Update geojson with tooltip data
        tk_lookup = filtered_tk.set_index('ISO3')[['Import Partner', 'Percent', 'Amount (USD Millions)']].to_dict(orient='index')
        for feature in geojson['features']:
            iso = feature['id']
            if iso in tk_lookup:
                feature['properties']['Import Partner'] = tk_lookup[iso]['Import Partner']
                feature['properties']['Percent'] = tk_lookup[iso]['Percent']
                feature['properties']['Amount (USD Millions)'] = tk_lookup[iso]['Amount (USD Millions)']
            else:
                feature['properties']['Import Partner'] = 'N/A'
                feature['properties']['Percent'] = 'N/A'
                feature['properties']['Amount (USD Millions)'] = 'N/A'

        # Style function
        def style_function(feature):
            iso = feature['id']
            return {
                'fillColor': color_dict.get(iso, '#cccccc'),
                'color': 'black',
                'weight': 0.5,
                'fillOpacity': 0.8
            }

        # Create map (do all processing first, then clear and display)
        M = folium.Map(location=[20, 10], zoom_start=2)
        folium.GeoJson(
            geojson,
            style_function=style_function,
            tooltip=folium.GeoJsonTooltip(
                fields=['name', 'Import Partner', 'Percent', 'Amount (USD Millions)'],
                aliases=['Country', 'Top Import Source', 'Percent', 'Amount (USD Millions)'],
                localize=True
            )
        ).add_to(M)

        # Only clear and display after map is fully built
        clear_output(wait=True)
        display(M)

# Set event handler
slider.observe(update_map, names='value')

# Initial display
display(slider)
display(map_output)

# Trigger first map
update_map()

SelectionSlider(description='Period:', layout=Layout(width='90%'), options=('2000-01', '2000-02', '2000-03', '…

Output()

In [44]:
!pip install iso3166

In [49]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
import pandas as pd
from iso3166 import countries_by_alpha3, countries

# ISO3 to country name lookup dictionary
iso3_to_country = {code: country.name for code, country in countries_by_alpha3.items()}
iso2_to_iso3 = {country.alpha2: country.alpha3 for country in countries}

year_month_options = sorted(tk['TIME_PERIOD'].unique())

slider = widgets.SelectionSlider(
    options=year_month_options,
    description='Period:',
    orientation='horizontal',
    layout={'width': '90%'},
    style={'description_width': 'initial'}
)

map_output = widgets.Output()

def update_map_px(change=None):
    with map_output:
        year_month = slider.value
        year = int(year_month.split('-')[0])
        month = int(year_month.split('-')[1])

        filtered_tk = tk[(tk['Year'].astype(int) == year) & (tk['Month'].astype(int) == month)]
        if len(filtered_tk) == 0:
            clear_output(wait=True)
            print(f"No data available for {year_month}")
            return

        filtered_tk = filtered_tk.copy()
        filtered_tk['color_key'] = filtered_tk.apply(
            lambda row: color_grid.get((row['source_cat'], row['share_bin']), '#cccccc'), axis=1
        )

        # Map ISO2 to ISO3 for locations
        filtered_tk['ISO3_from_ISO2'] = filtered_tk['Importer_Code'].map(iso2_to_iso3).fillna(filtered_tk['Importer_Code'])

        # Map country names for hover
        filtered_tk['CountryName'] = filtered_tk['ISO3'].map(iso3_to_country).fillna(filtered_tk['ISO3'])

        fig = go.Figure()
        for color in filtered_tk['color_key'].unique():
            color_data = filtered_tk[filtered_tk['color_key'] == color]

            fig.add_trace(go.Choropleth(
                locations=color_data['ISO3_from_ISO2'],  # ISO3 codes here
                z=[1] * len(color_data),
                locationmode='ISO-3',
                colorscale=[[0, color], [1, color]],
                showscale=False,
                hovertemplate=(
                    '<b>%{customdata[0]}</b><br>' +
                    'Top Import Source: %{customdata[1]}<br>' +
                    'Percent: %{customdata[2]}%<br>' +
                    'Amount: $%{customdata[3]}M<br>' +
                    '<extra></extra>'
                ),
                customdata=color_data[['CountryName', 'Import Partner', 'Percent', 'Amount (USD Millions)']].values,
                marker_line_color='black',
                marker_line_width=0.5,
                name=''
            ))

        fig.update_layout(
            title=f'Import Data - {year_month}',
            geo=dict(
                projection_type='robinson',
                showframe=False,
                showcoastlines=True,
                showland=True,
                landcolor='black',
                showocean=True,
                oceancolor='lightgray',
                bgcolor='white'
            ),
            height=600,
            margin=dict(t=50, b=0, l=0, r=0),
            showlegend=False
        )

        clear_output(wait=True)
        fig.show()

slider.observe(update_map_px, names='value')
display(slider)
display(map_output)
update_map_px()


SelectionSlider(description='Period:', layout=Layout(width='90%'), options=('2000-01', '2000-02', '2000-03', '…

Output()